In [8]:
%%capture
! pip install -r requirements.txt

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import shutil
import json

In [10]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = "cuda" if torch.cuda.is_available() else "cpu". # -- Use this line if have CUDA GPU
device

device(type='mps')

In [11]:
# -- Create the file structure
def make_file_structure():
    base_dir = "."
    dirs = ["images/train", "images/val", "labels/train", "labels/val"]

    for d in dirs:
        os.makedirs(os.path.join(base_dir, d), exist_ok=True)

# -- Add data to images folder
def add_images_to_file(src, dest, start=0, end=None, train=True):
    src_path = os.path.join(src, "data")

    if train:
        dest_path = os.path.join(dest, "train")
    else:
        dest_path = os.path.join(dest, "val")

    # -- Remove existing folder from previous run and make a new one
    if os.path.exists(dest_path):
        shutil.rmtree(dest_path)
    os.makedirs(dest_path, exist_ok=True)

    # Collecting all the image files
    files = [f for f in os.listdir(src_path) if f.endswith((".jpg", ".png"))]
    files = sorted(files)
    
    # If end is None, process all files
    if end is None:
        end = len(files)
    
    files = files[start:end]

    # -- Copying the selected files
    for file in files:
        shutil.copy(os.path.join(src_path, file), os.path.join(dest_path, file))
    
    print(f"Copied {len(files)} images to {dest_path}")
    return files 

# -- Adding labels to label folder
def add_labels_to_file(src, dest, image_files=None, train=True):
    json_file = os.path.join(src, "coco.json")

    with open(json_file, 'r') as f:
        data = json.load(f)
    
    annotations = data["annotations"]
    images = {img["id"]: img for img in data["images"]}

    # Create a mapping from category_id to index
    categories = {cat["id"]: idx for idx, cat in enumerate(data["categories"])}

    # Create destination directory if it doesn't exist
    label_dir = os.path.join(dest, "train" if train else "val")
    os.makedirs(label_dir, exist_ok=True)
    
    # If image_files is provided, filter to only those images
    if image_files is not None:
        image_filenames = set(image_files)
        # Filter images dict to only include the specified files
        images = {img_id: img_info for img_id, img_info in images.items() 
                  if os.path.basename(img_info["file_name"]) in image_filenames}
    
    # Group annotations by image_id (only for filtered images)
    img_anno = {}
    for anno in annotations:
        img_id = anno["image_id"]
        if img_id in images:
            if img_id not in img_anno:
                img_anno[img_id] = []
            img_anno[img_id].append(anno)
    
    # Process each image
    processed = 0
    for img_id, annos in img_anno.items():
        img_info = images[img_id]
        img_width = img_info["width"]
        img_height = img_info["height"]
        img_filename = img_info["file_name"]

        # Extract just the filename
        base_filename = os.path.basename(img_filename)
        
        # Create label files
        label_filename = os.path.splitext(base_filename)[0] + ".txt"
        label_path = os.path.join(dest, "train" if train else "val", label_filename)

        # Write in YOLO format
        with open(label_path, "w") as l:
            for anno in annos:
                category_id = anno["category_id"]
                
                if category_id not in categories:
                    print(f"Warning: Category {category_id} not found")
                    continue
                    
                class_idx = categories[category_id]

                bbox = anno["bbox"]

                x, y, w, h = bbox
                x_center = (x + w / 2) / img_width
                y_center = (y + h / 2) / img_height
                w_norm = w / img_width
                h_norm = h / img_height

                l.write(f"{class_idx} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
        
        processed += 1
    
    print(f"Processed {processed} label files in {label_dir}")


make_file_structure()

# Try with the first 10 images for both train and val
# -- Mess around with end to change number of images processed
train_files = add_images_to_file("./images_thermal_train", "./images", start=0, end=10, train=True)
add_labels_to_file("./images_thermal_train", "./labels", image_files=train_files, train=True)

val_files = add_images_to_file("./images_thermal_val", "./images", start=0, end=10, train=False)
add_labels_to_file("./images_thermal_val", "./labels", image_files=val_files, train=False)

Copied 10 images to ./images/train
Processed 10 label files in ./labels/train
Copied 10 images to ./images/val
Processed 10 label files in ./labels/val


In [12]:
# Visualizing the first image and annotations

# import cv2

# first_img = cv2.imread('./images/train/' + train_files[0])
# plt.imshow(cv2.cvtColor(first_img, cv2.COLOR_BGR2RGB))

# first_img_anot_path = "./labels/train/" + os.listdir("./labels/train/")[0]

# with open(first_img_anot_path, 'r') as f:
#     annot = f.readlines()
#     for cls, xc, yx, w, h in [line.strip().split() for line in annot]:
#         cv2.rectangle(first_img, 
#                       (int((float(xc) - float(w)/2) * first_img.shape[1]), int((float(yx) - float(h)/2) * first_img.shape[0])),
#                       (int((float(xc) + float(w)/2) * first_img.shape[1]), int((float(yx) + float(h)/2) * first_img.shape[0])),
#                       (255, 0, 0), 2)

# plt.imshow(cv2.cvtColor(first_img, cv2.COLOR_BGR2RGB))

# Model

In [14]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

''' model.train(
    data="thermal.yaml",
    imgsz=640,
    epochs=100,
    batch=16,
    device=device,   # pass it here instead of using .to()
    hsv_h=0, hsv_s=0, hsv_v=0,  # no color augment for thermal
    degrees=0, shear=0, perspective=0,
) '''

model.train(data="config.yml", epochs=50, imgsz=640, batch=16, device=device)


New https://pypi.org/project/ultralytics/8.3.222 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=config.yml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretraine

train: Scanning /Users/FreemanYiu/Downloads/AVL_Thermal/labels/train.cache... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1908.8±600.5 MB/s, size: 104.2 KB)



val: Scanning /Users/FreemanYiu/Downloads/AVL_Thermal/labels/val.cache... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<?, ?it/s]

Plotting labels to runs/detect/train2/labels.jpg... 


...

                   all         10        200     0.0371       0.13     0.0734     0.0468



50 epochs completed in 0.049 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.12.11 torch-2.7.1 MPS (Apple M4 Pro)
Model summary (fused): 72 layers, 3,008,573 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.43it/s]


                   all         10        200        0.2    0.00175      0.101     0.0908
                person         10         51          0          0          0          0
                  bike          4          6          0          0          0          0
                   car         10        114          1    0.00877      0.504      0.454
                   dog          4          4          0          0          0          0
            skateboard         10         25          0          0          0          0
Speed: 0.4ms preprocess, 6.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x3dc9d5460>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
      